In [ ]:
!pip install deepface opencv-python-headless

In [ ]:
# Install necessary libraries
!pip install opencv-python-headless deepface matplotlib
import cv2
from deepface import DeepFace
import matplotlib.pyplot as plt
from PIL import Image
from google.colab.patches import cv2_imshow
import numpy as np
import urllib

In [ ]:
!pip install mediapipe
import mediapipe as mp

In [ ]:
from deepface import DeepFace
import numpy as np
from PIL import Image
import urllib.request
import cv2
from google.colab.patches import cv2_imshow
from IPython.display import display, Javascript, Image as IPImage
import base64
from io import BytesIO

# Emotion-based responses with GIFs
emotion_responses = {
    "sad": {
        "message": "I know it's tough. Please smile, things will get better! 😊",
        "image": "https://media.giphy.com/media/6Uqa3vLCikV1yNRmdU/giphy.gif",
    },
    "happy": {
        "message": "You're doing great! Keep the positivity going! 🎉",
        "image": "https://media.giphy.com/media/OvpA7YvA9INE7bgGOy/giphy.gif",
    },
    "angry": {
        "message": "Take a deep breath and let go of the stress. You’ve got this! 💪",
        "image": "https://media.giphy.com/media/vqvmGLZdqxNzCtVlR3/giphy.gif",
    },
    "surprise": {
        "message": "Wow, something amazing must be happening! Enjoy the moment! 🎊",
        "image": "https://media.giphy.com/media/cKfkw1pp9R5SX7hDl1/giphy.gif",
    },
    "fear": {
        "message": "It's okay to feel scared. You are stronger than you think! 🌟",
        "image": "https://media.giphy.com/media/sNPeJFq6YNEvLZdcqX/giphy.gif",
    },
    "disgust": {
        "message": "Shake it off! Better things are waiting for you! 🌈",
        "image": "https://media.giphy.com/media/v1.Y2lkPTc5MGI3NjExYnNudjAzNm11d2h1NmJmdW55cW9sM2Q0bWFxdzhnZHEzaHp6ejRreCZlcD12MV9naWZzX3NlYXJjaCZjdD1n/OrWYyUFkBhxlDPdmoy/giphy.gif",
    },
    "neutral": {
        "message": "Just breathe and take things one step at a time. You've got this! 😊",
        "image": "https://media.giphy.com/media/9CGOSBk40bQfLrfBsf/giphy.gif",
    }
}

# Function to capture image from webcam and display
def capture_image_from_webcam():
    display(Javascript("""
        (function() {
            navigator.mediaDevices.getUserMedia({ video: true })
            .then(function(stream) {
                var video = document.createElement('video');
                video.srcObject = stream;
                video.play();
                document.body.appendChild(video);
                var captureBtn = document.createElement('button');
                captureBtn.innerHTML = 'Capture Image';
                document.body.appendChild(captureBtn);
                captureBtn.addEventListener('click', function() {
                    var canvas = document.createElement('canvas');
                    canvas.width = video.videoWidth;
                    canvas.height = video.videoHeight;
                    canvas.getContext('2d').drawImage(video, 0, 0);
                    var imageData = canvas.toDataURL('image/png');
                    google.colab.kernel.invokeFunction('notebook.capture_image', [imageData], {});
                    stream.getTracks().forEach(track => track.stop());
                    video.remove();
                    captureBtn.remove();
                });
            });
        })();
    """))

# Function to capture image and analyze emotion
def capture_image(image_data):
    # Decode the base64 image data
    img_data = image_data.split(",")[1]
    img = Image.open(BytesIO(base64.b64decode(img_data)))
    img = np.array(img)

    # Preprocess image if needed
    if img.shape[2] != 3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Analyze the emotions in the frame using DeepFace
    try:
        analysis = DeepFace.analyze(img, actions=['emotion'], enforce_detection=False)
        dominant_emotion = analysis[0]['dominant_emotion']
        print(f"Dominant Emotion: {dominant_emotion}")
    except Exception as e:
        print(f"Error: {e}")
        dominant_emotion = "neutral"

    # Display the emotion-based message and GIF
    display_emotion_message(dominant_emotion)

    # Show the captured image using OpenCV
    cv2_imshow(img)

# Function to display the emotion-based message and GIF
def display_emotion_message(emotion):
    response = emotion_responses.get(emotion, emotion_responses["neutral"])

    # Display message and GIF
    print(response["message"])
    display(IPImage(url=response["image"]))

# Register the capture image function in Colab
output.register_callback('notebook.capture_image', capture_image)

# Trigger webcam capture
capture_image_from_webcam()
